# ResponseEvaluator

This notebook uses the `ResponseEvaluator` module to measure if the response from a query engine matches any response nodes. This is useful for measuring if the response was hallucinated. The data is extracted from the [New York City](https://en.wikipedia.org/wiki/New_York_City) wikipedia page.

In [1]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

In [2]:
# configuring logger to INFO level
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
from llama_index import (
    TreeIndex,
    VectorStoreIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    Response,
)
from llama_index.llms import OpenAI
from llama_index.evaluation import ResponseEvaluator
import pandas as pd

pd.set_option("display.max_colwidth", 0)

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


Using GPT-4 here for evaluation

In [4]:
# gpt-4
gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

evaluator_gpt4 = ResponseEvaluator(service_context=service_context_gpt4)

In [5]:
documents = SimpleDirectoryReader("./test_wiki_data/").load_data()

In [7]:
# create tree index
tree_index = TreeIndex.from_documents(documents=documents)

INFO:llama_index.indices.common_tree.base:> Building index from nodes: 3 chunks
> Building index from nodes: 3 chunks


In [8]:
# create vector index
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=ServiceContext.from_defaults(chunk_size=512)
)

In [9]:
# define jupyter display function


def display_eval_df(response: Response, eval_result: str) -> None:
    if response.source_nodes == []:
        print("no response!")
        return
    eval_df = pd.DataFrame(
        {
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": eval_result,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

To run evaluations you can call the `.evaluate()` function on the `Response` object return from the query to run the evaluations. Lets evaluate the outputs of both the tree_index and vector_index to see if there is any difference.

In [36]:
query_engine = tree_index.as_query_engine()
response_tree = query_engine.query(
    "How did New York City get its name?"
)
eval_result = evaluator_gpt4.evaluate(response_tree)

INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 1] Selected node: [4]/[4]
>[Level 1] Selected node: [4]/[4]


In [37]:
display_eval_df(response_tree, eval_result)

,Response,Source,Evaluation Result
0,"New York City got its name after the Duke of York (the future King James II and VII), who was given part of the colony by proprietors George Carteret and John Berkeley. The settlement was promptly renamed ""New York"" after the Duke of York.","settlement was promptly renamed ""New York"" after the Duke of York (the future King James II and VII), who would eventually be deposed in the Glorious Revolution. After the founding, the duke gave part of the colony to proprietors George Carteret and John Berkeley. Fort Orange, 150 miles (240 km) north on the Hudson River, was renamed Albany after James's Scottish title. The transfer was confirmed in 1667 by the Treaty of Breda, which concluded the Second Anglo-Dutch War.On August 24, 1673, during the Third Anglo-Dutch War, Dutch captain Anthony Colve seized the colony of New York from the English at the behest of Cornelis Evertsen the Youngest and rechristened it ""New Orange"" after William III, the Prince of Orange. The Dutch would soon return the island to England under the Treaty of Westminster of November 1674.Several intertribal wars among the Native Americans and some epidemics brought on by contact with the Europeans caused sizeable population losses for the Lenape between the ye...",YES


In [38]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query(
    "How did New York City get its name?"
)
eval_result = evaluator_gpt4.evaluate(response_vector)

In [39]:
display_eval_df(response_vector, eval_result)

,Response,Source,Evaluation Result
0,"New York City was named in honor of the Duke of York, who would become King James II of England. In 1664, King Charles II appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control. The city was then renamed New York in his honor.","a US$1 billion research and education center as a leader the climate crisis. == Etymology == In 1664, New York was named in honor of the Duke of York, who would become King James II of England. James's elder brother, King Charles II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control. == History == === Early history === In the pre-Columbian era, the area of present-day New York City was inhabited by Algonquian Native Americans, including the Lenape. Their homeland, known as Lenapehoking, included the present-day areas of Staten Island, Manhattan, the Bronx, the western portion of Long Island (including the areas that would later become the boroughs of Brooklyn and Queens), and the Lower Hudson Valley.The first documented visit into New York Harbor by a European was in 1524 by Italian Giovanni da Verrazzano, an explorer from Florence in the service of the French crown. He claim...",YES


## Benchmark on Generated Question

Now lets generate a few more questions so that we have more to evaluate with and run a small benchmark. In practic

In [19]:
from llama_index.evaluation import DatasetGenerator

question_generator = DatasetGenerator.from_documents(documents)
eval_questions = question_generator.generate_questions_from_nodes(5)

eval_questions

chunk_size_limit is deprecated, please specify chunk_size instead


['What is the population of New York City as of 2020?',
 'Which borough of New York City has the highest population?',
 'What is the economic significance of New York City?',
 'How did New York City get its name?',
 'What is the significance of the Statue of Liberty in New York City?']

In [22]:
import asyncio


def evaluate_query_engine(query_engine, questions):
    c = [query_engine.aquery(q) for q in questions]
    results = asyncio.run(asyncio.gather(*c))
    print("finished query")

    total_correct = 0
    for r in results:
        # evaluate with gpt 4
        eval_result = 1 if evaluator_gpt4.evaluate(r) == "YES" else 0
        total_correct += eval_result

    return total_correct, len(results)

In [23]:
vector_query_engine = vector_index.as_query_engine()
correct, total = evaluate_query_engine(vector_query_engine, eval_questions[:5])

print(f"score: {correct}/{total}")

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=681 request_id=f587e4f416ad0d995a4400bc0a3be400 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=681 request_id=f587e4f416ad0d995a4400bc0a3be400 response_code=200
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1128 request_id=9af1677bef53dc66d8824900c8958b27 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1128 request_id=9af1677bef53dc66d8824900c8958b27 response_code=200
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2529 request_id=a58de27cf52038d452c2df03dd9251cb response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2529 request_id=a58de27cf52038d452c2df03dd9251cb response_code=200
INFO:openai:message='OpenAI API response' 

In [25]:
tree_query_engine = tree_index.as_query_engine()
correct, total = evaluate_query_engine(tree_query_engine, eval_questions[:5])

print(f"score: {correct}/{total}")

INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 1] Selected node: [1]/[1]
>[Level 1] Selected node: [1]/[1]
INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 1] Selected node: [2]/[2]
>[Level 1] Selected node: [2]/[2]
INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 0] Selected node: [3]/[3]
>[Level 0] Selected node: [3]/[3]
INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 1] Selected node: [5]/[5]
>[Level 1] Selected node: [5]/[5]
INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 1] Selected node: [4]/[4]
>[Level 1] Selected node: [4]/[4]
INFO:llama_index.indices.tree.select_lea